In [2]:
import sys
sys.path.append("/home/paperspace/fastai/")
import matplotlib
matplotlib.use('Agg')

from tqdm import tqdm
tqdm.monitor_interval = 0

from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
from numpy import genfromtxt

In [4]:
train_targs = genfromtxt('train_targs.csv')
val_targs = genfromtxt('targs.csv')

train_targs_T = V(train_targs)
val_targs_T = V(val_targs)

train_targs_TL = train_targs_T.type(torch.cuda.LongTensor)
val_targs_TL = val_targs_T.type(torch.cuda.LongTensor)

In [ ]:
p1 = genfromtxt('Parth_ResNext_512FC_Train.csv', delimiter=',')
p1v = genfromtxt('Parth_ResNext_512FC.csv', delimiter=',')

In [ ]:
p2 = genfromtxt('Arka_ResNext_Old_128FC_Train.csv', delimiter=',')
p2v = genfromtxt('Arka_ResNext_Old_128FC.csv', delimiter=',')

In [ ]:
p3 = genfromtxt('Arka_ResNext_128FC_Train.csv', delimiter=',')
p3v = genfromtxt('Arka_ResNext_128FC.csv', delimiter=',')

In [ ]:
# p4 = genfromtxt('Arka_4_ResNext_128FC_train.csv', delimiter=',')
# p4v = genfromtxt('Arka_4_ResNext_128FC.csv', delimiter=',')

In [ ]:
# p5 = genfromtxt('Arka_5_ResNext_128FC_train.csv', delimiter=',')
# p5v = genfromtxt('Arka_5_ResNext_128FC.csv', delimiter=',')

In [ ]:
# p6 = genfromtxt('Aditya_6_Inc_128FC_train.csv', delimiter=',')
# p6v = genfromtxt('Aditya_6_Inc_128FC.csv', delimiter=',')

In [ ]:
# p7 = genfromtxt('Aditya_7_Inc_128FC_train.csv', delimiter=',')
# p7v = genfromtxt('Aditya_7_Inc_128FC.csv', delimiter=',')

In [ ]:
# p8 = genfromtxt('Aditya_8_IncRes_128FC_train.csv', delimiter=',')
# p8v = genfromtxt('Aditya_8_IncRes_128FC.csv', delimiter=',')

In [ ]:
p12345678 = np.concatenate((p1, p2, p3, p4, p5, p6, p7, p8), axis=1)
p12345678v = np.concatenate((p1v, p2v, p3v, p4v, p5v, p6v, p7v, p8v), axis=1)

In [ ]:
p45678 = np.concatenate((p4, p5, p6, p7, p8), axis=1)

In [ ]:
p45678v = np.concatenate((p4v, p5v, p6v, p7v, p8v), axis=1)

In [ ]:
# df_train = pd.DataFrame(p12345678)
df_val = pd.DataFrame(p12345678v)

In [ ]:
# df_train.to_csv('ifood_train_12345678.csv', header=False, index=False)
df_val.to_csv('ifood_val_12345678.csv', header=False, index=False)

In [5]:
p12345678 = genfromtxt('ifood_train_12345678.csv', delimiter=',')
p12345678v = genfromtxt('ifood_val_12345678.csv', delimiter=',')

In [ ]:
p45678 = genfromtxt('ifood_train_45678.csv', delimiter=',')
p45678v = genfromtxt('ifood_val_45678.csv', delimiter=',')

In [6]:
def accuracy_np(preds, targs):
    preds = np.argmax(preds, 1)
    return (preds==targs).mean()

def accuracytop3_np(preds, targs):
    preds_3 = np.argsort(preds, axis=1)[:, -3:]
    return ((preds_3[:, 0] == targs) + (preds_3[:, 1] == targs) +
            (preds_3[:, 2] == targs)).mean()

In [ ]:
p12345678v.shape

In [ ]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return x


In [ ]:
model = torch.nn.Sequential(nn.Conv2d(8, 16, kernel_size=(3, 1)), 
                            nn.Conv2d(16, 32, kernel_size=(3, 1)), 
                            nn.Conv2d(32, 64, kernel_size=(3,1)), Flatten(),
                           nn.Linear(13120, 211), nn.LogSoftmax())
model.cuda()

In [ ]:
inp_dat = V(p1245678)
inp_dat2 = inp_dat.view(-1, 8, 211, 1)

In [ ]:
o1 = model(inp_dat2)

In [ ]:
o1.shape

In [ ]:
class CSVDataset(Dataset):
    def __init__(self, csvfname):
        super().__init__()
        self.a = genfromtxt('csvfname', delimiter=',')
    def __len__(self):
        return len(self.a)
    def __getitem__(self, idx):
        return self.a[idx]


In [8]:
import torch
from numpy import genfromtxt
model = torch.nn.Sequential(torch.nn.Linear(in_features=1688, out_features=211), torch.nn.LogSoftmax())
model = model.cuda()

inp_dat = V(p12345678)
inp_dat2 = inp_dat.view(-1, 8, 211, 1)
inp_dat22 = inp_dat2[100,:,:,:]
val_dat = V(p12345678v)
val_dat2 = val_dat.view(-1, 8, 211, 1)
val_ = inp_dat2[100,:,:,:]

# out = model(inp_dat)

# opt = torch.optim.Adam(model.parameters(), lr=1e-3)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
crit = nn.NLLLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'min')
num_epoch=1501
val_outs = train1(num_epoch, opt, model, inp_dat2, val_dat2, train_targs_TL, val_targs_TL)
# for ep in raze(num_epoch):
#     opt.zero_grad()
#     outs = model(inp_dat)
#     loss_l = crit(outs, train_targs_TL)
#     loss_l.backward()
#     opt.step()
#     val_outs = model(val_dat)
#     loss_v = crit(val_outs, val_targs_TL)
#     scheduler.step(loss_v.data[0])
#     if ep % 500 == 0:
#         print(ep, loss_l.data, loss_v.data)

RuntimeError: cuda runtime error (2) : out of memory at /pytorch/torch/lib/THC/generic/THCStorage.cu:58

In [7]:
def train1(num_epoch, opt, model, inp_dat, val_dat, train_targs_TL, val_targs_TL):
    for ep in range(num_epoch):
        opt.zero_grad()
        outs = model(inp_dat)
        loss_l = crit(outs, train_targs_TL)
        loss_l.backward()
        opt.step()
        val_outs = model(val_dat)
        loss_v = crit(val_outs, val_targs_TL)
#         scheduler.step(loss_v.data[0])
        if ep % 500 == 0:
            print(ep, loss_l.data, loss_v.data)
#             scheduler.step(loss_v.data[0])
    return val_outs

In [ ]:
loss_v.data[0]

In [ ]:
val_outs_np = val_outs.data.cpu().numpy()

print(accuracy_np(val_outs_np, val_targs))

accuracytop3_np(val_outs_np, val_targs)